In [1]:
import pandas as pd

In [2]:
filename = "/home/ran/datasets/test.parquet"
df = pd.read_parquet(filename)

In [3]:
df.head()

,image_id,image_tensor,label
0,img_0,b'PK\x03\x04\x00\x00\x08\x08\x00\x00\x00\x00\x...,0
1,img_1,b'PK\x03\x04\x00\x00\x08\x08\x00\x00\x00\x00\x...,0
2,img_2,b'PK\x03\x04\x00\x00\x08\x08\x00\x00\x00\x00\x...,0
3,img_3,b'PK\x03\x04\x00\x00\x08\x08\x00\x00\x00\x00\x...,1
4,img_4,b'PK\x03\x04\x00\x00\x08\x08\x00\x00\x00\x00\x...,1


In [4]:
import pandas as pd
import torch
from io import BytesIO


def load_dataset_from_dataframe(file_path):
    """
    Load the dataset from a DataFrame stored in a Parquet file.

    Args:
        file_path (str): Path to the Parquet file.

    Returns:
        pd.DataFrame: Loaded DataFrame.
    """
    df = pd.read_parquet(file_path, engine="pyarrow")

    # Convert image tensor bytes back to tensors
    def bytes_to_tensor(image_bytes):
        buffer = BytesIO(image_bytes)
        return torch.load(buffer)

    df["image_tensor"] = df["image_tensor"].apply(bytes_to_tensor)
    return df

In [5]:
df = load_dataset_from_dataframe(filename)
print(df.head())

/tmp/ipykernel_254653/1638804816.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(buffer)


  image_id                                       image_tensor  label
0    img_0  [[[tensor(0.9059), tensor(0.9373), tensor(0.91...      0
1    img_1  [[[tensor(0.9725), tensor(0.9529), tensor(0.76...      0
2    img_2  [[[tensor(0.6431), tensor(0.4392), tensor(0.39...      0
3    img_3  [[[tensor(0.6980), tensor(0.3882), tensor(0.35...      1
4    img_4  [[[tensor(0.7412), tensor(0.8039), tensor(0.86...      1


In [7]:
df["image_tensor"][0]

tensor([[[0.9059, 0.9373, 0.9137,  ..., 0.9608, 0.9490, 0.9412],
         [0.9451, 0.9882, 0.9843,  ..., 0.9529, 0.9490, 0.9412],
         [0.9608, 0.9608, 0.9294,  ..., 0.9529, 0.9451, 0.9412],
         ...,
         [0.9098, 0.7725, 0.3608,  ..., 0.9608, 0.9647, 0.9647],
         [0.9686, 0.7137, 0.6863,  ..., 0.9608, 0.9608, 0.9647],
         [0.6000, 1.0000, 0.9686,  ..., 0.9608, 0.9608, 0.9647]],

        [[0.8706, 0.8824, 0.8235,  ..., 0.9451, 0.9569, 0.9608],
         [0.9176, 0.9451, 0.9020,  ..., 0.9490, 0.9569, 0.9608],
         [0.9412, 0.9216, 0.8667,  ..., 0.9490, 0.9529, 0.9608],
         ...,
         [0.6980, 0.5608, 0.1529,  ..., 0.9608, 0.9647, 0.9647],
         [0.7961, 0.5451, 0.5333,  ..., 0.9608, 0.9608, 0.9647],
         [0.4667, 0.9490, 0.8627,  ..., 0.9608, 0.9608, 0.9647]],

        [[0.8431, 0.8706, 0.8471,  ..., 0.9412, 0.9451, 0.9451],
         [0.8941, 0.9373, 0.9216,  ..., 0.9412, 0.9451, 0.9451],
         [0.9255, 0.9176, 0.8784,  ..., 0.9412, 0.9412, 0.